In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #3")
conf.set("spark.master", "local[*]")

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/20 01:38:58 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/20 01:38:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 01:39:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/transfer_cost.txt

--2024-06-20 01:39:08--  https://s3-geospatial.s3.us-west-2.amazonaws.com/transfer_cost.txt
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.92.147.2, 52.218.250.1, 52.92.241.242, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.92.147.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286779 (280K) [text/plain]
Saving to: ‘transfer_cost.txt’

transfer_cost.txt   100%[===================>] 280.06K   591KB/s    in 0.5s    

2024-06-20 01:39:09 (591 KB/s) - ‘transfer_cost.txt’ saved [286779/286779]



In [4]:
!ls -tl

total 660
-rw-r--r-- 1 sangwon sangwon  26357 Jun 20 01:38 PySpark_DataFrame_2.ipynb
drwxr-xr-x 2 sangwon sangwon   4096 Jun 20 01:38 spark-warehouse
-rw-r--r-- 1 sangwon sangwon  32855 Jun 20 01:30 PySpark_DataFrame_1.ipynb
-rw-r--r-- 1 sangwon sangwon  13964 Jun 20 01:17 PySpark_DataFrame_3.ipynb
-rw-r--r-- 1 sangwon sangwon  31613 Jun 20 01:17 PySpark_DataFrame_4.ipynb
-rw-r--r-- 1 sangwon sangwon  21447 Jun 20 01:17 PySpark_DataFrame_5.ipynb
-rw-r--r-- 1 sangwon sangwon  26539 Jun 20 01:17 PySpark_설치_및_테스트.ipynb
-rw-r--r-- 1 sangwon sangwon 286779 Apr 25  2022 transfer_cost.txt
-rw-r--r-- 1 sangwon sangwon 146855 Apr 10  2022 customer-orders.csv
-rw-r--r-- 1 sangwon sangwon  62728 Apr 10  2022 1800.csv


In [5]:
!head -5 transfer_cost.txt

On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling
On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today
On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today


In [6]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

schema = StructType([ StructField("text", StringType(), True)])
transfer_cost_df = spark.read.schema(schema).text("transfer_cost.txt")

In [7]:
transfer_cost_df.show(truncate=False)

+---------------------------------------------------------------------------+
|text                                                                       |
+---------------------------------------------------------------------------+
|On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85012 is $18.98 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85013 is 26.64 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85020 is 26.34 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85021 is $20.15 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85002 to 85001 is 21.57 at 

In [8]:
from pyspark.sql.functions import *
regex_str = r'On (\S+) the cost per ton from (\d+) to (\d+) is (\S+) at (.*)'

df_with_new_columns = transfer_cost_df\
    .withColumn('week', regexp_extract('text', regex_str, 1))\
    .withColumn('departure_zipcode', regexp_extract(column('text'), regex_str, 2))\
    .withColumn('arrival_zipcode', regexp_extract(transfer_cost_df.text, regex_str, 3))\
    .withColumn('cost', regexp_extract(col('text'), regex_str, 4))\
    .withColumn('vendor', regexp_extract(col('text'), regex_str, 5))

In [9]:
df_with_new_columns.printSchema()

root
 |-- text: string (nullable = true)
 |-- week: string (nullable = true)
 |-- departure_zipcode: string (nullable = true)
 |-- arrival_zipcode: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- vendor: string (nullable = true)



In [10]:
final_df = df_with_new_columns.drop("text")

In [16]:
final_df.write.csv("extracted.csv")

In [17]:
!ls -tl

total 664
drwxr-xr-x 2 sangwon sangwon   4096 Jun 20 01:42 extracted.csv
-rw-r--r-- 1 sangwon sangwon  26357 Jun 20 01:38 PySpark_DataFrame_2.ipynb
drwxr-xr-x 2 sangwon sangwon   4096 Jun 20 01:38 spark-warehouse
-rw-r--r-- 1 sangwon sangwon  32855 Jun 20 01:30 PySpark_DataFrame_1.ipynb
-rw-r--r-- 1 sangwon sangwon  13964 Jun 20 01:17 PySpark_DataFrame_3.ipynb
-rw-r--r-- 1 sangwon sangwon  31613 Jun 20 01:17 PySpark_DataFrame_4.ipynb
-rw-r--r-- 1 sangwon sangwon  21447 Jun 20 01:17 PySpark_DataFrame_5.ipynb
-rw-r--r-- 1 sangwon sangwon  26539 Jun 20 01:17 PySpark_설치_및_테스트.ipynb
-rw-r--r-- 1 sangwon sangwon 286779 Apr 25  2022 transfer_cost.txt
-rw-r--r-- 1 sangwon sangwon 146855 Apr 10  2022 customer-orders.csv
-rw-r--r-- 1 sangwon sangwon  62728 Apr 10  2022 1800.csv


In [18]:
!ls -tl extracted.csv/

total 156
-rw-r--r-- 1 sangwon sangwon      0 Jun 20 01:42 _SUCCESS
-rw-r--r-- 1 sangwon sangwon 156423 Jun 20 01:42 part-00000-1c53739d-409a-40b4-bfbb-a5644f5f1826-c000.csv


In [19]:
!head -5 extracted.csv/part-00000-1c53739d-409a-40b4-bfbb-a5644f5f1826-c000.csv

2021-01-04,85001,85002,$28.32,ABC Hauling
2021-01-04,85001,85004,$25.68,ABC Hauling
2021-01-04,85001,85007,19.86,ABC Hauling
2021-01-04,85001,85007,20.52,Haul Today
2021-01-04,85001,85010,20.72,Haul Today


In [20]:
final_df.write.format("json").save("extracted.json")

In [21]:
!ls -tl extracted.json/

total 428
-rw-r--r-- 1 sangwon sangwon      0 Jun 20 01:43 _SUCCESS
-rw-r--r-- 1 sangwon sangwon 436305 Jun 20 01:43 part-00000-3c67d891-88d0-418f-b520-6a8170356eaf-c000.json


In [22]:
!head -1 extracted.json/part-00000-3c67d891-88d0-418f-b520-6a8170356eaf-c000.json

{"week":"2021-01-04","departure_zipcode":"85001","arrival_zipcode":"85002","cost":"$28.32","vendor":"ABC Hauling"}
